______

# Land Cover Classification for use in the CAPRA Model 

## Image Classification
_____

### Learning Objectives

In this lesson you will learn concepts of supervised classification. You will explore processes of training data collection, classifier selection, classifier training, image classification and accuracy assessment. 

Image classification means mapping the values captured by remote sensors that are encoded as image digital levels to specific land cover types. <br>
Classifying remotely sensed data into a thematic map is a relevant task beacuse the resulting information is the basis for many environmental and socioeconomic applications. In this example, a crop map type is produced by applying a [*supervised*](https://en.wikipedia.org/wiki/Supervised_learning) [*Random Forest*](https://en.wikipedia.org/wiki/Random_forest) algorithm. 

### Supervised Classification

A supervised classification method consists of training a classifier (algorithm) using (ground) truth data to classify  subsequently unseen data. <br>
The process flow in a supervised classification includes basically, the following steps:
* Collect the ground data. This data is used to train the classifier and validate its results.
* Train the classifier.
* Apply the classifier to produce a classified image.
* Assess the classification accuracy.

In [2]:
import ee
from IPython.display import Image
ee.Initialize()
# Load the image
# load the image collection and filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

CBD = ee.FeatureCollection("USDOS/LSIB/2013")
boundary = CBD.filterMetadata('name', 'equals', 'DOMINICAN REPUBLIC')

image = ee.Image(l8sr
            .filterDate('2019-01-01', '2019-04-30')
            .filterBounds(boundary)
            .sort('CLOUD_COVER')   
            .first()
)

For creating simple cloud-free Landsat composites, Earth Engine provides the ee.Algorithms.Landsat.simpleComposite() method. This method selects a subset of scenes at each location, converts to TOA reflectance, applies the simple cloud score and takes the median of the least cloudy pixels. This example creates a simple composite using default parameters and compares it to a composite using custom parameters for the cloud score threshold and the percentile:

In [ ]:
import pprint
# load the image collection and filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2018-01-01', '2018-12-31').filterBounds(boundary)                      

composite = ee.Algorithms.Landsat.simpleComposite(l8sr)
pprint(composite.getInfo())

In [3]:
import pprint
# Load tables into feature collection
#points = ee.FeatureCollection('ft:10q302Kafv_V_Le-jm1Cj274Ejz0kTsSWTKNxf3br').remap([1, 2, 3,4], [0, 1, 2, 3], "class")
#points = ee.FeatureCollection('ft:1p-YuR8JdqopYb-LrUxb43xzGHQyKmhGr1EJU3wku').remap([1, 2, 3,4], [0, 1, 2, 3], "name")
points = ee.FeatureCollection('ft:1Axs89HgE3yIgPoTMxTjL965OYgtBrp2i1O2waz29')
pprint.pprint(points.getInfo())

{'columns': {'description': 'String', 'landcover': 'String', 'name': 'String'},
 'features': [{'geometry': {'coordinates': [-71.621951, 18.453311],
                            'type': 'Point'},
               'id': '2',
               'properties': {'description': '', 'landcover': '4', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-71.620277, 18.453611],
                            'type': 'Point'},
               'id': '3',
               'properties': {'description': '', 'landcover': '4', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-71.619972, 18.454308],
                            'type': 'Point'},
               'id': '4',
               'properties': {'description': '', 'landcover': '4', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-71.624655, 18.453962],
                            'type': 'Point'},
               'id': '5',
               

               'type': 'Feature'},
              {'geometry': {'coordinates': [-71.167808, 19.09389],
                            'type': 'Point'},
               'id': '142',
               'properties': {'description': '', 'landcover': '1', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-71.165587, 19.094234],
                            'type': 'Point'},
               'id': '143',
               'properties': {'description': '', 'landcover': '1', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-71.171895, 19.093849],
                            'type': 'Point'},
               'id': '144',
               'properties': {'description': '', 'landcover': '1', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-71.168119, 19.090899],
                            'type': 'Point'},
               'id': '145',
               'properties': {'description': '', 'lan

                            'type': 'Point'},
               'id': '230',
               'properties': {'description': '', 'landcover': '2', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-69.827766, 19.156131],
                            'type': 'Point'},
               'id': '231',
               'properties': {'description': '', 'landcover': '2', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-69.840748, 19.155441],
                            'type': 'Point'},
               'id': '232',
               'properties': {'description': '', 'landcover': '2', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates': [-69.837872, 19.157387],
                            'type': 'Point'},
               'id': '233',
               'properties': {'description': '', 'landcover': '2', 'name': ''},
               'type': 'Feature'},
              {'geometry': {'coordinates

In [4]:
# Select the bands for training
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [5]:
# Sample the input imagery to get a FeatureCollection of training data.
training = image.select(bands).sampleRegions(points,['landcover'],30)
pprint.pprint(training.getInfo())

{'columns': {},
 'features': [],
 'properties': {'band_order': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']},
 'type': 'FeatureCollection'}


In [ ]:
# Make a Random Forest classifier and train it.
rf = ee.Classifier.randomForest(10).train(training,["landcover"],bands)
print(rf)

In [ ]:
# Classify the input imagery.
result = image.select(bands).classify(rf)
print(result)

In [ ]:
region = boundary.geometry().bounds().getInfo()['coordinates'][0]
task = ee.batch.Export.image.toDrive(
  image = result,
  description = "LCimage",
  #assetId = 'users/ximenamesa/LC' + "DR",
  maxPixels = 100000000,   
  #region = region,
  scale = 2)
task.start()
print("Task started")

Classification result could be improved by adding more data such as vegetation indices and textural features. You may try to perform a new classification considering those features.